In [92]:
# Importamos las bibliotecas que utilizaremos:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import joblib

In [93]:
# Leemos el archivo modelo_prediccion.parquet:
modelo_prediccion = pd.read_parquet('../data_post_etl/prediccion/modelo_prediccion.parquet')

In [94]:
# Creamos un TfidfVectorizer y transformamos 'predictor' en una matriz TF-IDF:

matriz = TfidfVectorizer(min_df=2, max_df=0.93, ngram_range=(1, 2), max_features=34000, 
dtype=np.float32).fit_transform(modelo_prediccion['predictor'])

In [95]:
# Creamos y aplicamos la reducción de dimensionalidad a la matriz utilizando SVD en una sola línea:
matriz_reducida = TruncatedSVD(n_components=350, random_state=42).fit_transform(matriz)

In [ ]:
def recomendar_pelicula(titulo_pelicula):
    # Encontramos el índice de la película:
    indice = modelo.index[modelo['title'].str.lower() == titulo_pelicula.lower()].tolist()
    if not indice:
        return "Película no encontrada"
    
    # Calculamos la matriz de similitud del coseno y obtenemos los puntajes:
    puntajes_similitud = cosine_similarity(matriz_reducida[indice[0]].reshape(1, -1), matriz_reducida)
    
    # Ordenamos las películas basadas en la similitud:
    puntajes_similitud = sorted(enumerate(puntajes_similitud[0]), key=lambda x: x[1], reverse=True)
    
    # Obtenemos los títulos de las películas más similares:
    indices_peliculas = [i[0] for i in puntajes_similitud[1:6]]  # 5 películas más similares
    return {"Recomendaciones": modelo['title'].iloc[indices_peliculas].tolist()}

{'Recomendaciones': ['toy story 2', 'hawaiian vacation', 'small fry', 'toy story 3', 'partysaurus rex']}


In [ ]:
# Guardamos la matriz TF-IDF como 'matriz.pkl':
joblib.dump(matriz, os.path.join('../data_post_etl/prediccion/', 'matriz.pkl'))

# Guardamos la matriz reducida como 'matriz_reducida.pkl':
joblib.dump(matriz_reducida, os.path.join('../data_post_etl/prediccion/', 'matriz_reducida.pkl'))

['../data_post_etl/prediccion/matriz_reducida.pkl']